In [17]:
import pandas as pd
import numpy as np

File filled manually based on MSCI factsheets and other research documents, most notably from:

https://www.researchgate.net/figure/Composition-of-the-MSCI-All-Country-World-Index-1987-2012-7_tbl1_325426617

In [11]:
country_weights = pd.read_csv('country_weights.csv', index_col=0, parse_dates=True)
df_weights = country_weights.T
df_weights.index = pd.DatetimeIndex(df_weights.index)
df_weights

C:\Users\lukas\AppData\Local\Temp\ipykernel_29096\537021929.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  country_weights = pd.read_csv('country_weights.csv', index_col=0, parse_dates=True)


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Sweden,...,Peru,Hungary,Czechia,Egypt,Colombia,Pakistan,Jordan,Sri Lanka,Venezuela,Marocco
1998-01-01,46.56,11.26,10.40,2.29,4.10,3.40,4.20,1.22,1.5,1.20,...,0.08,0.08,0.06,0.00,0.06,0.05,0.01,0.01,0.10,0.00
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00,NaN,0.04,0.01,0.01,0.08,0.00
2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00,NaN,0.03,0.01,0.01,0.06,0.00
2001-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.01,NaN,0.03,0.01,0.00,0.04,0.01
2002-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.01,NaN,0.02,0.01,0.00,0.02,0.01
2003-01-01,53.99,8.44,11.50,2.20,3.10,3.10,2.60,1.88,1.6,1.20,...,0.02,0.05,0.02,0.01,0.00,0.01,0.01,0.00,0.01,0.01
2004-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.02,NaN,0.01,0.01,0.00,0.01,0.01
2005-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.03,NaN,0.01,0.01,0.00,0.01,0.01
2006-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.05,NaN,0.02,0.01,0.00,0.00,0.02
2007-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.07,NaN,0.02,0.01,0.00,0.00,0.02


Skips some countries that I filled manually to keep years when they were included and excluded from indexes

In [13]:
countries_to_fill = df_weights.columns[df_weights.isna().sum(axis=0) > 0]
countries_to_fill

Index(['United States', 'Japan', 'United Kingdom', 'Canada', 'France',
       'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Sweden', 'Spain',
       'Hong Kong', 'Italy', 'Singapore', 'Denmark', 'Finland', 'Belgium',
       'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria', 'Portugal',
       'China', 'Taiwan', 'India', 'Korea', 'Brazil', 'Russia', 'South Africa',
       'Mexico', 'Malaysia', 'Thailand', 'Indonesia', 'Turkiye', 'Chile',
       'Poland', 'Greece', 'Philippines', 'Peru', 'Hungary', 'Czechia',
       'Colombia'],
      dtype='object')

Given country weights in some years fill the rest with progressive values

In [36]:
def fill_with_linspace_values(start_year, end_year):
    for coutry in countries_to_fill:
        start_value = df_weights.loc[start_year, coutry]
        end_value = df_weights.loc[end_year, coutry]    
        period_to_cover = int(end_year) - int(start_year) + 1
        
        df_weights.loc[start_year:end_year, coutry] = np.linspace(
            start_value, end_value, period_to_cover).reshape(period_to_cover)

In [42]:
fill_with_linspace_values('2021','2023')
df_weights

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Sweden,...,Peru,Hungary,Czechia,Egypt,Colombia,Pakistan,Jordan,Sri Lanka,Venezuela,Marocco
1998-01-01,46.56000,11.2600,10.4000,2.290,4.10,3.4000,4.200,1.22000,1.5000,1.20,...,0.0800,0.08000,0.06000,0.00,0.0600,0.05,0.01,0.01,0.10,0.00
1999-01-01,48.04600,10.6960,10.6200,2.272,3.90,3.3400,3.880,1.35200,1.5200,1.20,...,0.0680,0.07400,0.05200,0.00,0.0480,0.04,0.01,0.01,0.08,0.00
2000-01-01,49.53200,10.1320,10.8400,2.254,3.70,3.2800,3.560,1.48400,1.5400,1.20,...,0.0560,0.06800,0.04400,0.00,0.0360,0.03,0.01,0.01,0.06,0.00
2001-01-01,51.01800,9.5680,11.0600,2.236,3.50,3.2200,3.240,1.61600,1.5600,1.20,...,0.0440,0.06200,0.03600,0.01,0.0240,0.03,0.01,0.00,0.04,0.01
2002-01-01,52.50400,9.0040,11.2800,2.218,3.30,3.1600,2.920,1.74800,1.5800,1.20,...,0.0320,0.05600,0.02800,0.01,0.0120,0.02,0.01,0.00,0.02,0.01
2003-01-01,53.99000,8.4400,11.5000,2.200,3.10,3.1000,2.600,1.88000,1.6000,1.20,...,0.0200,0.05000,0.02000,0.01,0.0000,0.01,0.01,0.00,0.01,0.01
2004-01-01,51.55200,8.4720,11.1200,2.494,3.32,3.3000,2.820,2.06200,1.5400,1.22,...,0.0300,0.05800,0.03400,0.02,0.0060,0.01,0.01,0.00,0.01,0.01
2005-01-01,49.11400,8.5040,10.7400,2.788,3.54,3.5000,3.040,2.24400,1.4800,1.24,...,0.0400,0.06600,0.04800,0.03,0.0120,0.01,0.01,0.00,0.01,0.01
2006-01-01,46.67600,8.5360,10.3600,3.082,3.76,3.7000,3.260,2.42600,1.4200,1.26,...,0.0500,0.07400,0.06200,0.05,0.0180,0.02,0.01,0.00,0.00,0.02
2007-01-01,44.23800,8.5680,9.9800,3.376,3.98,3.9000,3.480,2.60800,1.3600,1.28,...,0.0600,0.08200,0.07600,0.07,0.0240,0.02,0.01,0.00,0.00,0.02


In [78]:
# Manual fixes
df_weights.loc['2022', 'Russia'] = 0
df_weights.loc['2003', 'Colombia'] = 0.01

Normalize to match 100

In [72]:
df_weights.sum(axis=1)

1998-01-01    99.63
1999-01-01    99.63
2000-01-01    99.64
2001-01-01    99.73
2002-01-01    99.78
2003-01-01    99.87
2004-01-01    99.81
2005-01-01    99.76
2006-01-01    99.74
2007-01-01    99.70
2008-01-01    99.67
2009-01-01    99.61
2010-01-01    99.59
2011-01-01    99.59
2012-01-01    99.61
2013-01-01    99.61
2014-01-01    99.70
2015-01-01    99.68
2016-01-01    99.63
2017-01-01    99.60
2018-01-01    99.57
2019-01-01    99.87
2020-01-01    99.91
2021-01-01    99.94
2022-01-01    99.72
2023-01-01    99.87
2024-01-01    99.99
dtype: float64

In [73]:
df_weights_norm = df_weights.multiply((100/df_weights.sum(axis=1)), axis=0)

In [74]:
df_weights_norm = df_weights_norm.round(2)

In [75]:
df_weights_norm.sum(axis=1)

1998-01-01     99.96
1999-01-01     99.96
2000-01-01     99.97
2001-01-01     99.97
2002-01-01     99.97
2003-01-01     99.96
2004-01-01     99.98
2005-01-01     99.98
2006-01-01     99.96
2007-01-01     99.95
2008-01-01     99.95
2009-01-01     99.98
2010-01-01     99.98
2011-01-01     99.97
2012-01-01     99.95
2013-01-01     99.96
2014-01-01     99.96
2015-01-01     99.95
2016-01-01     99.95
2017-01-01     99.94
2018-01-01     99.96
2019-01-01     99.96
2020-01-01     99.97
2021-01-01     99.98
2022-01-01     99.99
2023-01-01     99.97
2024-01-01    100.00
dtype: float64

In [80]:
df_weights_norm = df_weights_norm['1999':]
df_weights_norm

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Sweden,...,Peru,Hungary,Czechia,Egypt,Colombia,Pakistan,Jordan,Sri Lanka,Venezuela,Marocco
1999-01-01,48.23,10.74,10.66,2.28,3.91,3.35,3.89,1.36,1.53,1.20,...,0.07,0.07,0.05,0.00,0.05,0.04,0.01,0.01,0.08,0.00
2000-01-01,49.71,10.17,10.88,2.26,3.71,3.29,3.57,1.49,1.55,1.20,...,0.06,0.07,0.04,0.00,0.04,0.03,0.01,0.01,0.06,0.00
2001-01-01,51.16,9.60,11.09,2.25,3.51,3.23,3.25,1.62,1.56,1.20,...,0.04,0.06,0.04,0.01,0.02,0.03,0.01,0.00,0.04,0.01
2002-01-01,52.62,9.02,11.30,2.22,3.31,3.17,2.93,1.75,1.58,1.20,...,0.03,0.06,0.03,0.01,0.01,0.02,0.01,0.00,0.02,0.01
2003-01-01,54.06,8.45,11.51,2.20,3.10,3.10,2.60,1.88,1.60,1.20,...,0.02,0.05,0.02,0.01,0.01,0.01,0.01,0.00,0.01,0.01
2004-01-01,51.65,8.49,11.14,2.49,3.33,3.31,2.83,2.06,1.54,1.22,...,0.03,0.06,0.03,0.02,0.01,0.01,0.01,0.00,0.01,0.01
2005-01-01,49.23,8.52,10.77,2.80,3.55,3.51,3.05,2.25,1.48,1.24,...,0.04,0.07,0.05,0.03,0.01,0.01,0.01,0.00,0.01,0.01
2006-01-01,46.80,8.56,10.39,3.09,3.77,3.71,3.27,2.44,1.42,1.26,...,0.05,0.07,0.06,0.05,0.02,0.02,0.01,0.00,0.00,0.02
2007-01-01,44.37,8.60,10.01,3.39,3.99,3.91,3.49,2.62,1.36,1.28,...,0.06,0.08,0.08,0.07,0.02,0.02,0.01,0.00,0.00,0.02
2008-01-01,41.94,8.63,9.63,3.68,4.21,4.11,3.71,2.80,1.30,1.30,...,0.07,0.09,0.09,0.09,0.03,0.02,0.01,0.00,0.00,0.03


Save to CSV

In [81]:
df_weights_norm.to_csv('acwi_coutry_weights.csv')